# 🩺 Medlock AI: Health Summary Notebook
This notebook lets you:
- Use AI to summarize patient text
- Save and retrieve from a local SQLite database
- Update and delete entries


In [ ]:
!pip install transformers

In [ ]:

import sqlite3
from datetime import datetime
from transformers import pipeline


In [ ]:

# Connect to DB
conn = sqlite3.connect("medlock_records.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS patient_summaries (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        full_input TEXT,
        summary TEXT,
        timestamp TEXT
    )
""")
conn.commit()

# Load summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

print("✅ Database and summarizer ready.")


In [ ]:

# 🔹 Summarize patient input
patient_input = """I've been experiencing shortness of breath, dizziness, and fatigue for 3 days."""
summary = summarizer(patient_input, max_length=60, min_length=25, do_sample=False)[0]['summary_text']

print("---- Summary ----")
print(summary)


In [ ]:

# 🔹 Save to DB
timestamp = datetime.now().isoformat()

cursor.execute("""
    INSERT INTO patient_summaries (full_input, summary, timestamp)
    VALUES (?, ?, ?)
""", (patient_input, summary, timestamp))
conn.commit()

print("✅ Summary saved to database.")


In [ ]:

# 🔹 Search DB
search_term = "fatigue"  # Change as needed

cursor.execute("""
    SELECT * FROM patient_summaries
    WHERE lower(full_input) LIKE ? OR lower(summary) LIKE ?
""", (f"%{search_term.lower()}%", f"%{search_term.lower()}%"))

results = cursor.fetchall()

for row in results:
    print(f"ID: {row[0]}\nSummary: {row[2]}\nTime: {row[3]}\n")


In [ ]:

# 🔹 Update or Delete
record_id = 1  # Replace with the actual ID

# Example: Update
new_summary = "Updated summary: Patient has dizziness, breathlessness, and fatigue."
cursor.execute("UPDATE patient_summaries SET summary = ? WHERE id = ?", (new_summary, record_id))
conn.commit()
print("✅ Updated.")

# Example: Delete
# cursor.execute("DELETE FROM patient_summaries WHERE id = ?", (record_id,))
# conn.commit()
# print("❌ Deleted.")
